## Description
This is a notebook for debugging OTF_Tel

#### Make sure before you perform fft, you shift the frequency domain back into the quarant ...




# Section
<details>
  <summary>Click to expand!</summary>
  
  ## Heading
  1. [Generate a PSF](#Generate-a-PSF)
    * [Zero Padding](#Zero-Padding)
    * [Setting up the system](#Setting-up-the-system)
    * [Pupil Function](#Pupil-Function)
    
  2.[Minimisation](#Minimisation)
    
  3.[How to fix the differences?](#How-to-fix-the-differences?)
    
  3.[Doesnt match with the IDL output](#Doesnt-match)
  
</details>

In [ ]:
# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
#Change to your path
os.chdir("/Users/alau/Repo/amiral")
from amiral import instructment, utils, parameter, gradient
from scipy.optimize import minimize 
from scipy import signal
%matplotlib inline

In [ ]:
# Global variable
# Parameters to modify
SEED = -1234       # Seed for random numbers generation
FLUX = 5e8         # Object total intensity [e-]
READ_OUT_NOISE = 10. # CCD read-out noise standard-deviation [e-]
DATA_DIR = "/Users/alau/IDLWorkspace/Data/Vesta_OASIS/"
DATA_FITS = DATA_DIR + "2018-06-08T05_27_05.809.fits"
DIMENSION = 256 # Dimension of the image
LOOP = True

test_data_dir = "/Users/alau/Data/amiral_fits/"

In [ ]:
# Setup of the system
fits_img = fits.open(DATA_FITS)
fits_img.info() 

obj = fits_img[0].data

aosys = instructment.aoSystem(sampling = 2,diameter = 7., occ_ratio = 0.1, no_acutuator = 30, wavelength = 500, dimension = DIMENSION, resolution_rad = None)
fX, fY, freqnull = aosys.psd_frequency_array(DIMENSION, aosys.samp_factor[0])

pupil = aosys.get_pupil_plane()


pupil_IDL = fits.open(test_data_dir+"pupil_plane.fits")
pupil_IDL = pupil_IDL[0].data


diff = utils.view_diff(pupil, pupil_IDL)
utils.info(diff)

In [ ]:
otf_tel_abs = aosys.pupil_to_otf_tel(pupil, absolute = True)
otf_tel_real = aosys.pupil_to_otf_tel(pupil, absolute = False)

otf_tel_abs_IDL = fits.open(test_data_dir+"otf_tel_abs.fits")
otf_tel_abs_IDL = otf_tel_abs_IDL[0].data

otf_tel_real_IDL = fits.open(test_data_dir+"otf_tel_real.fits")
otf_tel_real_IDL = otf_tel_real_IDL[0].data


diff = utils.view_diff(otf_tel_abs,otf_tel_abs_IDL)
utils.info(diff)

plt.imshow(otf_tel_abs)

utils.info(tel_abs)
utils.info(otf_tel_abs_IDL)

In [ ]:
psd_halo = aosys.psd_residual_halo(0.15)
psd_halo_IDL = fits.open(test_data_dir+"psd_halo.fits")
psd_halo_IDL = psd_halo_IDL[0].data

diff = utils.view_diff(psd_halo, psd_halo_IDL)

print(np.sum(psd_halo)- np.sum(psd_halo_IDL))

In [ ]:
param = {
    "r0": 0.15,                  
    "background": 1e-10 ,      
    "amplitude": 1.8,       
    "ax": 0.05,                            
    "ay": 0.05, 
    "theta": 0., 
    "beta": 1.5,
    "mu": 1., 
    "rho0": 1., 
    "p": 1.
}

param_key, param_input = utils.dict2array(param)

psd_ao = aosys.psd_residual_ao (guess = param_input)







psd_halo = aosys.psd_residual_halo(r0=0.15)

psd = psd_halo + psd_ao 


psd_total_IDL = fits.open(test_data_dir+"psd_total.fits")
psd_total_IDL = psd_total_IDL[0].data
psd_halo_IDL = fits.open(test_data_dir+"psd_halo.fits")
psd_halo_IDL = psd_halo_IDL[0].data

diff = utils.view_diff(psd_ao,psd_total_IDL-psd_halo_IDL)
utils.info(diff)


plt.imshow(psd_ao)

print(np.max(np.abs(psd_halo_IDL-psd_halo)))

In [ ]:
integral, SR = aosys.psd_integral(psd, r0=param["r0"])
otf_atmo = aosys.otf_atmo(psd)
otf_total = aosys.otf_total(otf_tel_abs, otf_atmo)
psf_total = aosys.psfao(otf_total)

print("\nSum of the PSF (which should be excatly 1.)", np.sum(np.abs(psf_total)))
print("\nMax of the otf_atmo: (which should be excatly 1.)", np.max(otf_atmo))
print("\nMax of the otf_tel: (which should be excatly 1.)", np.max(otf_tel_abs))
print("\nMax of the OTF (which should be excatly 1.)", np.max(otf_total))

In [ ]:
# # Try to match the IDL defintion 
# size_pupil_array = int(np.ceil(DIMENSION/aosys.sampling))
# pupil_array = np.zeros((DIMENSION, DIMENSION))

# Rpup = np.fft.fftshift(utils.dist(size_pupil_array)/(size_pupil_array/2))
# Rpup_IDL = utils.load_fits("/Users/alau/Rpup.fits")

# fig, ax =  plt.subplots(figsize = (10,10))
# pos = ax.imshow(Rpup_IDL-Rpup_IDL)
# fig.colorbar(pos, ax=ax)
# # 
# utils.info(Rpup_IDL-Rpup_IDL)
# #pup_tf = np.fft.fftshift(np.abs(np.fft.ifft2(np.abs(np.fft.fft2(pupil_array))**2)))/np.sum(pupil_array)

In [ ]:
# pup_IDL = utils.load_fits("/Users/alau/pupil.fits")

# pupil_array[0:size_pupil_array,0:size_pupil_array] = (Rpup <= 1) * (Rpup >= aosys.occ_ratio)


# fig, ax =  plt.subplots(figsize = (10,10))
# pos = ax.imshow(pup_IDL-pupil_array)
# fig.colorbar(pos, ax=ax)

# utils.info(pup_IDL-pupil_array)
# print(np.sum(pup_IDL-pupil_array))

In [ ]:
diff = np.fft.fft2(pup_IDL)- np.fft.fft2(pupil_array)


fig, ax =  plt.subplots(figsize = (10,10))
pos = ax.imshow(np.real(diff))
fig.colorbar(pos, ax=ax)

utils.info(diff)

In [ ]:
# compare the abs(fft(pup))
abs_pup = np.abs(np.fft.fft2(pupil_array))/(512**2)
abs_pup_IDL = utils.load_fits("/Users/alau/abs_pup.fits")

fig, ax =  plt.subplots(figsize = (10,10))
pos = ax.imshow(np.log10(abs_pup-abs_pup_IDL))
ax.set_title('FFT Difference (IDL-python)')
fig.colorbar(pos, ax=ax)

utils.info(abs_pup-abs_pup_IDL)

In [ ]:
a = [1,2,3,4]

print(np.fft.fft(a)*(1/4))
print(np.fft.ifft(a)/(1/4))

In [ ]:
# continue working on the defintion of the pupil function
abs_fft_abs_pup2 = np.fft.fftshift(np.fft.ifft2(np.abs(np.fft.fft2(pupil_array))**2))
abs_fft_abs_pup2 = np.abs(abs_fft_abs_pup2)/(512**2)

abs_fft_abs_pup2_IDL = utils.load_fits("/Users/alau/abs_fft_abs_pup2.fits")


diff = abs_fft_abs_pup2_IDL-abs_fft_abs_pup2
utils.info(diff)

fig, ax =  plt.subplots(figsize = (10,10))
pos = ax.imshow(diff)
ax.set_title('Pupil Plane Difference (IDL-python)')
fig.colorbar(pos, ax=ax)



In [ ]:
# define pup_tf 
abs_fft_abs_pup2 = np.fft.fftshift(np.fft.ifft2(np.abs(np.fft.fft2(pupil_array))**2))

pup_tf = abs_fft_abs_pup2/np.sum(pupil_array)

pup_tf_real = np.real(pup_tf)

#utils.info(pup_tf, imaginary = True)

otf_tel_real_IDL = utils.load_fits("/Users/alau/otf_tel_real.fits")

diff = otf_tel_real_IDL - pup_tf_real
fig, ax =  plt.subplots(figsize = (10,10))
pos = ax.imshow(diff)
ax.set_title('Pupil Plane Difference (IDL-python)')


fig.colorbar(pos, ax=ax)


utils.info(diff)

In [ ]:
# Investigate the difference between the real one 
# No clue ...

otf_tel = aosys.pupil_to_otf_tel(pupil)

otf_tel_real_IDL = utils.load_fits("/Users/alau/otf_tel_real.fits")

fig, ax =  plt.subplots(figsize = (10,10))
pos = ax.imshow(np.real(otf_tel_real_IDL-otf_tel))
ax.set_title('Pupil Plane Difference (IDL-python)')
fig.colorbar(pos, ax=ax)

utils.info(diff)

## Check the total OTF

In [ ]:
oft_IDL = utils.load_fits("/Users/alau/otf_total_real.fits")

In [ ]:
diff = otf_total - oft_IDL
utils.info(diff)
fig, ax =  plt.subplots()
pos = ax.imshow(np.real(diff[256-80:256+80,256-80:256+80]))
ax.set_title('OTF total diff')
fig.colorbar(pos, ax=ax)

In [ ]:
a = np.fft.ifft2(np.abs(np.fft.fft2(pupil_array))**2)
plt.imshow(np.real(a))

Why the real part of otf_tel is so off?

In [ ]:
otf_tel = np.fft.fftshift(np.fft.ifft2(np.abs(np.fft.fft2(pupil_array))**2))

In [ ]:
IDL_fft_pup2 = utils.load_fits("/Users/alau/abs_fft_pup2.fits")
abs_fft_pup2 = np.abs(np.fft.fft2(pupil_array)*(1/512**2))**2
diff = utils.view_diff(IDL_fft_pup2, abs_fft_pup2)

utils.info(diff)

plt.imshow(np.log10(abs_fft_pup2))

In [ ]:
# compare real(fftshift(abs(fft(pup))^2,/INVERSE))
# continue working on the defintion of the pupil function
abs_fft_abs_pup2 = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift((np.abs(np.fft.fft2(pupil_array)*(1./512**2))**2)))/(1./512**2))
real_fft_abs_pup2 = np.real(abs_fft_abs_pup2)

real_fft_abs_pup2_IDL = utils.load_fits("/Users/alau/real_fft_abs_pup2.fits")


diff = real_fft_abs_pup2_IDL-real_fft_abs_pup2
utils.info(diff)

print(real_fft_abs_pup2_IDL.max()/real_fft_abs_pup2.max())

fig, ax =  plt.subplots(figsize = (10,10))
pos = ax.imshow(diff)
ax.set_title('Pupil Plane Difference (IDL-python)')
fig.colorbar(pos, ax=ax)




In [ ]:
# Check the difference between the real part of the fft using pupil as an example

In [ ]:
real_fft_pup_IDL = utils.load_fits("/Users/alau/real_fft_pup.fits")
real_fft_pup = np.real(np.fft.fft2(pupil_array))*(1./512**2)

diff = utils.view_diff(real_fft_pup_IDL,real_fft_pup)
utils.info(diff)

In [ ]:
# otf_tel

otf_tel = aosys.pupil_to_otf_tel(pupil_array)

otf_tel_test = real_fft_abs_pup2 / np.sum(pupil_array) * (512**2)

otf_tel_IDL_real = utils.load_fits("/Users/alau/otf_tel_real.fits")

diff = utils.view_diff(np.real(otf_tel_test),otf_tel_IDL_real)


utils.info(diff)


print(np.where( diff == diff.max()))